# Analysis and Visualization
This notebook is focused on the model in action and answers the question you posed or address the problem we are investigating. Includes plots and visualizations as necessary to address the question and/or research topic.

### Sandy Tam -- CNN Analysis and Visualization

In [ ]:
# import libraries

### Christina Vo -- Logistic Regression Analysis and Visualization

In [ ]:
# import libraries

### Conclusion and Results

share our findings and what we conclude as a team